In [115]:
import pprint
import re

In [116]:
with open("input.txt", "rt") as f:
    seeds, *maps = f.read().strip().split("\n\n")

seeds = list(map(int, seeds.split(": ")[1].split()))

_maps = []
for m in maps:
    _, *ranges = re.split("(?: map:)?\n", m)
    _maps.append([tuple(map(int, r.split())) for r in ranges])
maps = _maps

Part 1

In [117]:
locations = []
for seed in seeds:
    num = seed
    for map_ in maps:
        for dst, src, length in map_:
            if num >= src and num < src + length:
                num += (dst - src)
                break
    locations.append(num)
min(locations)
# locs

382895070

Part 2

In [118]:
def pairwise(iterable):
    it = iter(iterable)
    return zip(it, it)


# Convert seeds and maps into ranges represented as (start, stop, increment),
# where 'start' and 'stop' define the ends of the range (stop is exclusive),
# and 'increment' is the value by which all elements in the range need to be adjusted
seed_ranges = [(src, src + length, 0) for src, length in pairwise(seeds)]
tmaps = [[(src, src + length, dst - src) for dst, src, length in m] for m in maps]

# Iterate through each map transforming ranges
ranges = seed_ranges
ranges_to_check = ranges.copy()
for map_ranges in tmaps:
    overlaps = []

    for m_start, m_end, m_inc in map_ranges:
        # Iterate through each input range to check for overlaps
        ranges_left = []
        for r_start, r_end, r_inc in ranges_to_check:
            # If the input range doesn't overlap with the map range, skip
            if r_start > m_end or m_start > r_end:
                ranges_left.append((r_start, r_end, r_inc))
                continue

            # Find overlapping range
            o_start = max(r_start, m_start)
            o_end = min(r_end, m_end)
            o_inc = r_inc + m_inc
            overlaps.append((o_start, o_end, o_inc))

            # Find non-overlapping range(s)
            if o_start > r_start:
                ranges_left.append((r_start, o_start, r_inc))
            if o_end < r_end:
                ranges_left.append((o_end, r_end, r_inc))

        # I'm assuming that ranges in a map itself aren't overlapping
        # so for each range in a map, I'll check input ranges that
        # weren't already mapped by any previous ranges in this map
        ranges_to_check = ranges_left

    # Mapped ranges should be transformed, before checking the next map
    overlaps = [(start + inc, end + inc, 0) for start, end, inc in overlaps]

    # Next map should check all ranges produced by previous map
    # and all the ranges that weren't mapped
    ranges_to_check = overlaps + ranges_left

# If there are no more maps to go through - we have the result
mapped_ranges = ranges_to_check

min([start for start, *_ in mapped_ranges])

17729182